## Final Project --- the relationship between depression and other diseases and habits

**Part 1: data cleaning**

**Hailey  Wang**

# 1. import

## 1.1 import packages

In [1]:
import pandas as pd
import altair as alt
from scipy.stats import pearsonr
import geopandas as gpd
import numpy as np
from shapely import wkt

### 1.2 import raw CDC data

In [2]:
cdc_places = pd.read_csv('./raw_data.csv',
                        usecols=['Year', 'StateAbbr', 'StateDesc', 'LocationName', 'Data_Value', 'TotalPopulation', 'LocationID', 'Data_Value_Type', 'Short_Question_Text', 'Geolocation'])

cdc_places

,Year,StateAbbr,StateDesc,LocationName,Data_Value_Type,Data_Value,TotalPopulation,LocationID,Short_Question_Text,Geolocation
0,2021,GA,Georgia,Ware,Crude prevalence,4.6,36033,13299,Stroke,POINT (-82.4215072 31.050881)
1,2021,IN,Indiana,Ohio,Crude prevalence,3.8,5978,18115,Stroke,POINT (-84.9642994 38.940527)
2,2021,IN,Indiana,Owen,Crude prevalence,4.0,21446,18119,Stroke,POINT (-86.8388453 39.3173394)
3,2021,IA,Iowa,Iowa,Crude prevalence,3.3,16568,19095,Stroke,POINT (-92.0591226 41.6839184)
4,2021,IA,Iowa,Marion,Crude prevalence,3.0,33380,19125,Stroke,POINT (-93.0938678 41.3314327)
...,...,...,...,...,...,...,...,...,...,...
228765,2021,WA,Washington,Lewis,Crude prevalence,9.6,84398,53041,Health Insurance,POINT (-122.3774443 46.5800714)
228766,2021,WI,Wisconsin,Door,Age-adjusted prevalence,22.0,30369,55029,Depression,POINT (-87.0486828 45.0934165)
228767,2021,WI,Wisconsin,Dodge,Crude prevalence,16.1,89313,55027,Current Smoking,POINT (-88.7019394 43.4296276)
228768,2021,WV,West Virginia,Cabell,Age-adjusted prevalence,18.4,93418,54011,Cognitive Disability,POINT (-82.2433966 38.4195778)


**Describe the database**

source: https://data.cdc.gov/500-Cities-Places/PLACES-Local-Data-for-Better-Health-County-Data-20/swc5-untb

time period: 2020/2021

what the data can and can't tell you: It tells the county level health indicates. This dataset includes estimates for 36 measures: 13 for health outcomes, 9 for preventive services use, 4 for chronic disease-related health risk behaviors, 7 for disabilities, and 3 for health status.

It has some data missing. For example, it has no depreesion rate for all counties in Florida. 

### 1.3 import income data

In [3]:
income = pd.read_csv('./new income county.csv')
income

,State,County,Income
0,Alabama,Autauga,"48,914"
1,Alabama,Baldwin,"55,865"
2,Alabama,Barbour,"40,795"
3,Alabama,Bibb,"37,175"
4,Alabama,Blount,"42,852"
...,...,...,...
3018,Wyoming,Sweetwater,"56,252"
3019,Wyoming,Teton,"362,522"
3020,Wyoming,Uinta,"44,358"
3021,Wyoming,Washakie,"54,898"


Describe the database

source: https://www.bea.gov/news/2022/personal-income-county-and-metropolitan-area-2021

time period: 2021 

What the data can and can’t tell you: It is personal income data from the Bureau of Economic Analysis (BEA). BEA’s County and metropolitan area per capita personal income statistics are calculated by dividing personal income with population. BEA used Census population figures to calculate annual per capita personal income statistics for 2020 through 2022.

# 2.clean CDC data 

## 2.1 check for missing data

In [4]:
cdc_places[
    cdc_places['LocationName'].isna()]

,Year,StateAbbr,StateDesc,LocationName,Data_Value_Type,Data_Value,TotalPopulation,LocationID,Short_Question_Text,Geolocation
18,2021,US,United States,NaN,Age-adjusted prevalence,2.8,331893745,59,Stroke,NaN
443,2021,US,United States,NaN,Crude prevalence,10.8,331893745,59,Health Insurance,NaN
958,2021,US,United States,NaN,Age-adjusted prevalence,16.7,331893745,59,Binge Drinking,NaN
1161,2021,US,United States,NaN,Crude prevalence,7.0,331893745,59,Cancer (except skin),NaN
1222,2021,US,United States,NaN,Crude prevalence,14.7,331893745,59,Mental Health,NaN
...,...,...,...,...,...,...,...,...,...,...
16314,2021,US,United States,NaN,Crude prevalence,15.5,331893745,59,Binge Drinking,NaN
17126,2021,US,United States,NaN,Age-adjusted prevalence,31.0,331893745,59,High Cholesterol,NaN
17140,2020,US,United States,NaN,Age-adjusted prevalence,83.7,331893745,59,Cervical Cancer Screening,NaN
17211,2021,US,United States,NaN,Crude prevalence,3.3,331893745,59,Stroke,NaN


**It seems like we are missing data of location name and geolocation for nation level data. It makes sense because there is not a specific location or geolocation for the whole nation. And we will not need nation level data later, so I am dropping the missing national data.**

In [5]:
cdc_places_drop_us = cdc_places.dropna(subset = ['LocationName'])
cdc_places_drop_us

,Year,StateAbbr,StateDesc,LocationName,Data_Value_Type,Data_Value,TotalPopulation,LocationID,Short_Question_Text,Geolocation
0,2021,GA,Georgia,Ware,Crude prevalence,4.6,36033,13299,Stroke,POINT (-82.4215072 31.050881)
1,2021,IN,Indiana,Ohio,Crude prevalence,3.8,5978,18115,Stroke,POINT (-84.9642994 38.940527)
2,2021,IN,Indiana,Owen,Crude prevalence,4.0,21446,18119,Stroke,POINT (-86.8388453 39.3173394)
3,2021,IA,Iowa,Iowa,Crude prevalence,3.3,16568,19095,Stroke,POINT (-92.0591226 41.6839184)
4,2021,IA,Iowa,Marion,Crude prevalence,3.0,33380,19125,Stroke,POINT (-93.0938678 41.3314327)
...,...,...,...,...,...,...,...,...,...,...
228765,2021,WA,Washington,Lewis,Crude prevalence,9.6,84398,53041,Health Insurance,POINT (-122.3774443 46.5800714)
228766,2021,WI,Wisconsin,Door,Age-adjusted prevalence,22.0,30369,55029,Depression,POINT (-87.0486828 45.0934165)
228767,2021,WI,Wisconsin,Dodge,Crude prevalence,16.1,89313,55027,Current Smoking,POINT (-88.7019394 43.4296276)
228768,2021,WV,West Virginia,Cabell,Age-adjusted prevalence,18.4,93418,54011,Cognitive Disability,POINT (-82.2433966 38.4195778)


**We dropped 74 rows that are nation level data.**

## 2.2 check data types

In [6]:
cdc_places_drop_us.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228696 entries, 0 to 228769
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Year                 228696 non-null  int64  
 1   StateAbbr            228696 non-null  object 
 2   StateDesc            228696 non-null  object 
 3   LocationName         228696 non-null  object 
 4   Data_Value_Type      228696 non-null  object 
 5   Data_Value           228696 non-null  float64
 6   TotalPopulation      228696 non-null  int64  
 7   LocationID           228696 non-null  int64  
 8   Short_Question_Text  228696 non-null  object 
 9   Geolocation          228696 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 19.2+ MB


**data type looks almost fine! But I want to change the geolocation's data type to geometry, so that I can play with the map later**

### 2.2.1 adjust data type

In [7]:

cdc_places_drop_us['Geolocation'] = cdc_places_drop_us['Geolocation'].apply(wkt.loads)
cdc_places_drop_us_geo = gpd.GeoDataFrame(cdc_places_drop_us, geometry='Geolocation')

/var/folders/_p/zmr502n520jcpyzkqqhb1h580000gn/T/ipykernel_9826/1960375873.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdc_places_drop_us['Geolocation'] = cdc_places_drop_us['Geolocation'].apply(wkt.loads)


In [8]:
cdc_places_drop_us_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 228696 entries, 0 to 228769
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype   
---  ------               --------------   -----   
 0   Year                 228696 non-null  int64   
 1   StateAbbr            228696 non-null  object  
 2   StateDesc            228696 non-null  object  
 3   LocationName         228696 non-null  object  
 4   Data_Value_Type      228696 non-null  object  
 5   Data_Value           228696 non-null  float64 
 6   TotalPopulation      228696 non-null  int64   
 7   LocationID           228696 non-null  int64   
 8   Short_Question_Text  228696 non-null  object  
 9   Geolocation          228696 non-null  geometry
dtypes: float64(1), geometry(1), int64(3), object(5)
memory usage: 19.2+ MB


## 2.3 rename columns

Break down each Columns. I have the years of dataset, state and state abbreviation, county, data value(%), population of each county, location ID, short question text(to tell which data value it is reffered to) and geolocation.
I am now renaming the columns to make it clearer. 
**Change StateDesc to State; LocationName to County; Data_value to DataValue; Short_Questoin_Text to Category.**

In [9]:
cdc_places_drop_us_geo = cdc_places_drop_us_geo.rename(columns = {'StateDesc':'State', 'LocationName':'County', 'Data_Value':'DataValue', 'Data_Value_Type':'MeasuresOfFrequency' ,'Short_Question_Text':'Category'})
cdc_places_drop_us_geo 

,Year,StateAbbr,State,County,MeasuresOfFrequency,DataValue,TotalPopulation,LocationID,Category,Geolocation
0,2021,GA,Georgia,Ware,Crude prevalence,4.6,36033,13299,Stroke,POINT (-82.42151 31.05088)
1,2021,IN,Indiana,Ohio,Crude prevalence,3.8,5978,18115,Stroke,POINT (-84.96430 38.94053)
2,2021,IN,Indiana,Owen,Crude prevalence,4.0,21446,18119,Stroke,POINT (-86.83885 39.31734)
3,2021,IA,Iowa,Iowa,Crude prevalence,3.3,16568,19095,Stroke,POINT (-92.05912 41.68392)
4,2021,IA,Iowa,Marion,Crude prevalence,3.0,33380,19125,Stroke,POINT (-93.09387 41.33143)
...,...,...,...,...,...,...,...,...,...,...
228765,2021,WA,Washington,Lewis,Crude prevalence,9.6,84398,53041,Health Insurance,POINT (-122.37744 46.58007)
228766,2021,WI,Wisconsin,Door,Age-adjusted prevalence,22.0,30369,55029,Depression,POINT (-87.04868 45.09342)
228767,2021,WI,Wisconsin,Dodge,Crude prevalence,16.1,89313,55027,Current Smoking,POINT (-88.70194 43.42963)
228768,2021,WV,West Virginia,Cabell,Age-adjusted prevalence,18.4,93418,54011,Cognitive Disability,POINT (-82.24340 38.41958)


## 2.4 look for duplicates

In [10]:
len(cdc_places_drop_us_geo)

228696

**It originally has 228696 rows. Now let's see if there are any duplicates.**

In [11]:
cdc_places_drop_us_geo_duplicate = cdc_places_drop_us_geo[cdc_places_drop_us_geo.duplicated(subset=['State', 'County', 'Category'])]
cdc_places_drop_us_geo_duplicate

,Year,StateAbbr,State,County,MeasuresOfFrequency,DataValue,TotalPopulation,LocationID,Category,Geolocation
318,2021,AL,Alabama,Madison,Crude prevalence,3.0,395211,1089,Stroke,POINT (-86.55108 34.76424)
334,2021,AL,Alabama,Calhoun,Crude prevalence,22.7,115972,1015,Depression,POINT (-85.82791 33.77052)
359,2021,AL,Alabama,Choctaw,Age-adjusted prevalence,13.8,12533,1023,Binge Drinking,POINT (-88.24889 31.99095)
368,2021,CA,California,Calaveras,Crude prevalence,9.9,46221,6009,Current Asthma,POINT (-120.55411 38.19107)
407,2021,AR,Arkansas,Clark,Crude prevalence,9.6,21321,5019,Independent Living Disability,POINT (-93.17621 34.05332)
...,...,...,...,...,...,...,...,...,...,...
228765,2021,WA,Washington,Lewis,Crude prevalence,9.6,84398,53041,Health Insurance,POINT (-122.37744 46.58007)
228766,2021,WI,Wisconsin,Door,Age-adjusted prevalence,22.0,30369,55029,Depression,POINT (-87.04868 45.09342)
228767,2021,WI,Wisconsin,Dodge,Crude prevalence,16.1,89313,55027,Current Smoking,POINT (-88.70194 43.42963)
228768,2021,WV,West Virginia,Cabell,Age-adjusted prevalence,18.4,93418,54011,Cognitive Disability,POINT (-82.24340 38.41958)


**There are 114348 duplicates. It's exactly half of the number. What are they?**

There are two types of measures of frequency. According to CDC Places'description, age-adjusted prevalence is the standardized data of crude-prevalence data by the direct method to the year 2000 standard U.S. population, distribution 22 with 95% confidence intervals and by demographic characteristics when feasible. We will use age-adjusted data here, so I'm dropping all of the crude-prevalence data.

In [12]:
mask = cdc_places_drop_us_geo['MeasuresOfFrequency'] == 'Crude prevalence'
cdc_places_cleaned = cdc_places_drop_us_geo[~mask]
cdc_places_cleaned

,Year,StateAbbr,State,County,MeasuresOfFrequency,DataValue,TotalPopulation,LocationID,Category,Geolocation
10,2021,CA,California,Calaveras,Age-adjusted prevalence,10.3,46221,6009,Current Asthma,POINT (-120.55411 38.19107)
11,2021,AL,Alabama,Fayette,Age-adjusted prevalence,6.9,16148,1057,Coronary Heart Disease,POINT (-87.76429 33.71616)
12,2021,AR,Arkansas,Benton,Age-adjusted prevalence,13.7,293692,5007,Cognitive Disability,POINT (-94.25630 36.33782)
13,2021,AL,Alabama,Calhoun,Age-adjusted prevalence,34.1,115972,1015,Physical Inactivity,POINT (-85.82791 33.77052)
16,2020,CA,California,Sacramento,Age-adjusted prevalence,38.2,1588921,6067,Core preventive services for older men,POINT (-121.34433 38.45014)
...,...,...,...,...,...,...,...,...,...,...
228763,2021,WY,Wyoming,Sublette,Age-adjusted prevalence,3.7,8697,56035,Vision Disability,POINT (-109.91617 42.76793)
228764,2021,WI,Wisconsin,Green Lake,Age-adjusted prevalence,28.1,19229,55047,High Blood Pressure,POINT (-88.97038 43.78025)
228766,2021,WI,Wisconsin,Door,Age-adjusted prevalence,22.0,30369,55029,Depression,POINT (-87.04868 45.09342)
228768,2021,WV,West Virginia,Cabell,Age-adjusted prevalence,18.4,93418,54011,Cognitive Disability,POINT (-82.24340 38.41958)


In [13]:
len(cdc_places_cleaned)

114348

**I dropped half of the data. With no duplicates,now I get 114348 rows left.**

In [14]:
cdc_places_cleaned.to_csv(r'/Users/Owner/Documents/school work/J233/cleaned cdc places with python.csv', index=False)

## 2.5 look for depression value

In [15]:
cdc_places_depression = cdc_places_cleaned[cdc_places_cleaned['Category'] == 'Depression']
cdc_places_depression.sort_values('StateAbbr')

,Year,StateAbbr,State,County,MeasuresOfFrequency,DataValue,TotalPopulation,LocationID,Category,Geolocation
3365,2021,AK,Alaska,Juneau,Age-adjusted prevalence,20.2,31973,2110,Depression,POINT (-134.17845 58.37291)
11914,2021,AK,Alaska,Denali,Age-adjusted prevalence,19.9,1593,2068,Depression,POINT (-150.02702 63.68204)
1924,2021,AK,Alaska,Northwest Arctic,Age-adjusted prevalence,20.3,7560,2188,Depression,POINT (-160.02109 67.00507)
11848,2021,AK,Alaska,Dillingham,Age-adjusted prevalence,20.3,4772,2070,Depression,POINT (-158.26712 59.54333)
7566,2021,AK,Alaska,Nome,Age-adjusted prevalence,19.8,9865,2180,Depression,POINT (-164.18891 64.78369)
...,...,...,...,...,...,...,...,...,...,...
216518,2021,WY,Wyoming,Sweetwater,Age-adjusted prevalence,20.2,41614,56037,Depression,POINT (-108.87568 41.66033)
221452,2021,WY,Wyoming,Platte,Age-adjusted prevalence,20.1,8699,56031,Depression,POINT (-104.95396 42.13159)
216316,2021,WY,Wyoming,Johnson,Age-adjusted prevalence,20.7,8623,56019,Depression,POINT (-106.58854 44.04405)
219614,2021,WY,Wyoming,Albany,Age-adjusted prevalence,21.4,37608,56001,Depression,POINT (-105.72188 41.66551)


# 3. clean income data

In [16]:
combine_depression_income = pd.merge(
    cdc_places_depression,
    income, 
    how='outer', 
    on=['State','County'],
    #validate='1:1' 
)

combine_depression_income

,Year,StateAbbr,State,County,MeasuresOfFrequency,DataValue,TotalPopulation,LocationID,Category,Geolocation,Income
0,2021.0,AL,Alabama,Cullman,Age-adjusted prevalence,25.2,89496.0,1043.0,Depression,POINT (-86.86927 34.13192),"46,762"
1,2021.0,AK,Alaska,Bristol Bay,Age-adjusted prevalence,18.8,838.0,2060.0,Depression,POINT (-156.96681 58.74166),NaN
2,2021.0,AL,Alabama,Calhoun,Age-adjusted prevalence,23.3,115972.0,1015.0,Depression,POINT (-85.82791 33.77052),"43,045"
3,2021.0,AL,Alabama,Conecuh,Age-adjusted prevalence,23.1,11328.0,1035.0,Depression,POINT (-86.98872 31.43093),"40,467"
4,2021.0,AL,Alabama,Etowah,Age-adjusted prevalence,25.0,103162.0,1055.0,Depression,POINT (-86.03426 34.04764),"42,571"
...,...,...,...,...,...,...,...,...,...,...,...
3259,NaN,NaN,Utah,Orange,NaN,NaN,NaN,NaN,NaN,None,"55,304"
3260,NaN,NaN,Utah,Orleans,NaN,NaN,NaN,NaN,NaN,None,"52,213"
3261,NaN,NaN,Utah,Rutland,NaN,NaN,NaN,NaN,NaN,None,"56,441"
3262,NaN,NaN,Utah,Windham,NaN,NaN,NaN,NaN,NaN,None,"55,590"


In [17]:
combine_depression_income.dropna()

,Year,StateAbbr,State,County,MeasuresOfFrequency,DataValue,TotalPopulation,LocationID,Category,Geolocation,Income
0,2021.0,AL,Alabama,Cullman,Age-adjusted prevalence,25.2,89496.0,1043.0,Depression,POINT (-86.86927 34.13192),"46,762"
2,2021.0,AL,Alabama,Calhoun,Age-adjusted prevalence,23.3,115972.0,1015.0,Depression,POINT (-85.82791 33.77052),"43,045"
3,2021.0,AL,Alabama,Conecuh,Age-adjusted prevalence,23.1,11328.0,1035.0,Depression,POINT (-86.98872 31.43093),"40,467"
4,2021.0,AL,Alabama,Etowah,Age-adjusted prevalence,25.0,103162.0,1055.0,Depression,POINT (-86.03426 34.04764),"42,571"
5,2021.0,AR,Arkansas,Lee,Age-adjusted prevalence,22.8,8619.0,5077.0,Depression,POINT (-90.77972 34.77950),"39,589"
...,...,...,...,...,...,...,...,...,...,...,...
3109,2021.0,WA,Washington,Jefferson,Age-adjusted prevalence,28.0,33605.0,53031.0,Depression,POINT (-123.52706 47.80571),"61,489"
3110,2021.0,WA,Washington,Grays Harbor,Age-adjusted prevalence,28.4,76841.0,53027.0,Depression,POINT (-123.82673 47.11373),"47,304"
3111,2021.0,WI,Wisconsin,Calumet,Age-adjusted prevalence,21.9,52539.0,55015.0,Depression,POINT (-88.21213 44.07841),"58,066"
3112,2021.0,WI,Wisconsin,Monroe,Age-adjusted prevalence,23.9,46193.0,55081.0,Depression,POINT (-90.61997 43.94518),"49,825"


The depression info is from CDC, and income data is from BEA. Because the datasets have different sizes with some counties missing in one or the other, there will be some gaps when merging. So, I'm just getting rid of those gaps.

In [18]:
combine_depression_income.to_csv(r'/Users/Owner/Documents/school work/J233/combine_depression_income with python.csv', index=False)